# Descision tree

## ID3 tree

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = 'IRIS.csv'

In [3]:
data = pd.read_csv(path)
data

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


查看数据有几种类型

In [4]:
data['species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

划分数据

In [5]:
Iris_setosa = data[data['species'].isin(['Iris-setosa'])]
Iris_virginica = data[data['species'].isin(['Iris-virginica'])]
Iris_versicolor = data[data['species'].isin(['Iris-versicolor'])]

Iris_setosa.shape, Iris_virginica.shape, Iris_versicolor.shape

((50, 5), (50, 5), (50, 5))

In [6]:
cols = data.shape[1]
x_train = np.array(data.iloc[:, 0:cols-1]) # 数据集
y_train = np.array(data.iloc[:, cols-1:cols]) # 标签集
data_train = data.values
x_train.shape, y_train.shape, data_train.shape

((150, 4), (150, 1), (150, 5))

字典统计不同类型标签的出现次数

In [7]:
def countLabel(y_train):
    classDict = {}
    for label in y_train:
        if label in classDict.keys():
            classDict[label] += 1
        else:
            classDict[label] = 1
    return classDict

找到当前集合中最多的标签

In [8]:
def majorClass(y_train):
    classDict = countLabel(y_train)
    #对字典进行降序排序,内置sorted(),key参数，用于指定在作比较之前，调用何种函数对列表元素进行处理
    classSort = sorted(classDict.items(), key = lambda x:x[1], reverse=True)
    return classSort[0][0]

In [9]:
a = majorClass(data_train[:, cols-1])
a

'Iris-setosa'

计算当前集合的信息熵，${P}_{k}$是当前集合样本$D$中第$k$类样本所占比例，${k}=1, 2, ..., {|y|}$。
$$ Ent\left(D\right)=-\sum\limits_{k=1}^{|y|}{P}_{k}{\log}_{2}{P}_{k}$$

In [10]:
def information_entropy(y):
    classDict = countLabel(y)
    dict2list = np.array(list(classDict.values()))
    p_k = dict2list/dict2list.sum()
    Ent = -(p_k*np.log2(p_k)).sum()
    return Ent

In [11]:
information_entropy(data_train[:, cols-1])

1.584962500721156

计算当前集合$D$某一离散属性$a$的信息增益，$a$有$V$个可能取值$\{a^1, a^2, ..., a^v\}$
$$Gain\left(D, a\right)=Ent\left(D\right)-\sum\limits_{v=1}^{V}\frac{|D^v|}{|D|}Ent\left(D^v\right)$$

对于连续属性$a$的信息增益，假定$a$在$D$上出现了$n$个不同的取值，将这些值从小到大排列，记为$\{a^1, a^2, ...,a^n\}$。基于划分点$t$可将$D$分为子集$D_t^-$和$D_t^+$。
$$T_a=\left\{\frac{a^i+a^{i+1}}{2}|{1\leq i\leq n-1}\right\}$$
$$Gain\left(D, a\right)=\max_{t\in T_a}Gain\left(D, a, t\right)=\max_{t\in T_a}\left\{Ent\left(D\right)-\sum\limits_{\lambda \in \{-, +\}}\frac{|D_t^{\lambda}|}{|D|}Ent\left(D_t^{\lambda}\right)\right\}$$
需要注意的是，与离散属性不同，若当前结点的划分属性是连续属性，该属性还可作为其后代结点的划分属性。

In [12]:
def information_gain(data, a):
    cols = data.shape[1]
    data_a = data[:, a]
    data_a_sort = np.unique(data_a) #属性a删除重复值并排序
    data_a_sort.sort()
    first = np.delete(data_a_sort, len(data_a_sort)-1) #删除最后一个取值
    second = np.delete(data_a_sort, 0) #删除第一个取值
    T = (first + second)/2 #求得划分点t
    data_section = np.take(data, [a, cols-1], axis=1) #获取数据a属性列和标签列
    
    #集合data的信息熵
    Ent = information_entropy(data[:, cols-1])
    
    #记录属性a的最小条件熵，初始值为该集合信息熵最大值log2(|D|)
    classDict = countLabel(data[:, cols-1])
    min = np.log2(len(classDict))
    divide=0
    for t in T:
        index_negative = data_section[:, 0]<t #子集Dt-的索引
        index_positive = ~index_negative #子集Dt+的索引
        negative = data_section[index_negative]
        positive = data_section[index_positive]
        
        sum = (negative.shape[0]/data_section.shape[0])*information_entropy(negative[:, 1])\
                +(positive.shape[0]/data_section.shape[0])*information_entropy(positive[:, 1])
        if min > sum:
            min = sum
            divide = t
            
    Gain = Ent-min
    return Gain, divide

In [13]:
information_gain(data_train, 0)

(0.5572326878069267, 5.55)

## 定义二叉树

In [14]:
class BTree(object):
    #初始化
    def __init__(self, feature=None, value=None, left=None, right=None):
        self.feature = feature #划分属性
        self.value = value    # 划分值
        self.left = left    # 左子树
        self.right = right  # 右子树
        
    #前序遍历
    def preorder(self):
        if self.data is not None:
            print(self.feature, self.value, end=' ')
        if self.left is not None:
            self.left.preorder()
        if self.right is not None:
            self.right.preorder()

递归创建决策树

In [15]:
def createTree(data):
    
    classDict = countLabel(data[:, data.shape[1]-1])
    #当前集合的类别相同，则返回该类叶节点
    if len(classDict) == 1:
        return list(classDict)[0]
    
    #当前集合在所有属性上取值相同，则返回样本数最多的类别叶节点
    data_feature = data[:, 0:cols-2]
    if (data_feature[0]==data_feature).astype(int).sum() == data_feature.shape[0]*data_feature.shape[1]:
        return majorClass(data[:, data.shape[1]-1])
    
    #求得每个属性的信息增益和对应划分点
    gain_list = np.array([])
    divide_list = np.array([])
    #求得每个属性的信息增益和对应划分点
    for a in range(data.shape[1]-1):
        g, d = information_gain(data, a)
        gain_list = np.append(gain_list, g)
        divide_list = np.append(divide_list, d)
        
    #求得最大信息增益索引
    index = np.where(gain_list == gain_list.max())
    index = index[0][0]
    
    node = BTree(index, divide_list[index])
    
    #当前集合划分为子集Dt-和Dt+
    i = data[:, index] <= divide_list[index]
    data_negative = data[i]
    data_positive = data[~i]
    
    node.left = createTree(data_negative)
    node.right = createTree(data_positive)
    
    return node

In [18]:
tree = createTree(data_train)
tree